In [ ]:
!pip install -q tqdm pyriksdagen ipywidgets plotly

In [ ]:
from pyparlaclarin.read import paragraph_iterator, speeches_with_name
from pyriksdagen.utils import protocol_iterators, download_corpus

from pathlib import Path
from queue import Queue
from lxml import etree
from tqdm.auto import tqdm

import progressbar
import pyriksdagen
import ipywidgets

import plotly.offline as pyo
import plotly.graph_objs as go
from typing import List, Tuple


import sqlite3
import re
import os

pyo.init_notebook_mode()


In [ ]:
parser = etree.XMLParser(remove_blank_text=True)

data_dir = Path('.').resolve() / 'data'

In [ ]:
if len([file for file in data_dir.iterdir() if file.is_file()]) < 25:
    print('Did not find metadata files. Downloading.')
    download_corpus(partitions=["persons"])

records_data_dir = data_dir / 'records'
if len([subdir for subdir in data_dir.iterdir() if subdir.is_dir()]) < 158:
    print('Did not find protocols files. Downloading.')
    download_corpus(partitions=["records"])


In [ ]:
protocols = list(sorted(protocol_iterators(corpus_root="data/", start=1899, end=1941)))
print(len(protocols))

In [ ]:
int(protocols[0].split('/')[1][:4])

In [ ]:
def prepare_roots(protocols):
    for protocol in protocols:
        year = int(protocol.split('/')[1][:4])
        yield etree.parse(protocol, parser).getroot(), year

In [ ]:
def process_root_queue(q: Queue):
    """
    TODO: Extract debate names
    TODO: Extract dates
    """
    while not q.empty():
        c, element, year = q.get()
        if (who:= element.get('who')) is not None:
            u_id = element.get([key for key in element.keys() if key.endswith('}id')][0])
            assert u_id
            prev = element.get('prev')
            nxt = element.get('next')
    
            text = '\n\n'.join(re.sub(r'\s+' ,' ', seg.text) for seg in element.getchildren())
            yield u_id, prev, nxt, text, who, year
        else:
            for child in element.getchildren():
                if (child.tag.endswith('note') or child.tag.endswith('seg')) and not bool(re.search(r'^\S+dag', child.text)):
                    continue
                q.put((c+1, child, year))
        


In [ ]:
def extract_all_utterances(protocols):
    q = Queue()
    for root, year in prepare_roots(protocols):
        q.put((0,root, year))
    yield from process_root_queue(q)
    

In [ ]:
all_utterances = []
for utterance in tqdm(extract_all_utterances(protocols), total=701_218): #total=5273785):
    all_utterances.append(utterance)

In [ ]:
from itertools import islice
def batched(iterable, n, *, strict=False):
    # batched('ABCDEFG', 2) → AB CD EF G
    if n < 1:
        raise ValueError('n must be at least one')
    iterator = iter(iterable)
    while batch := tuple(islice(iterator, n)):
        if strict and len(batch) != n:
            raise ValueError('batched(): incomplete batch')
        yield batch

In [ ]:

tmp_db = './tmp.db'
if os.path.exists(tmp_db):
    os.unlink(tmp_db)

with sqlite3.connect(tmp_db) as conn:
    cur = conn.cursor()
    cur.execute('CREATE TABLE utterance (id str primary key, prev text, next text, who text, year int)')
    # cur.execute("PRAGMA compile_options LIKE '%SQLITE_ENABLE_FTS5%';")
    cur.execute('CREATE VIRTUAL TABLE utterance_fts USING fts5(content)')
    cur.execute('CREATE index next_index on utterance(next)')
    cur.execute('CREATE index prev_index on utterance(prev)')
    cur.execute('CREATE index who_index on utterance(who)')
    cur.execute('CREATE index year_index on utterance(year)')

    data = []
    for batch in tqdm(batched(all_utterances, 50_000), total=len(all_utterances)//50_000):
        data = [{'id':u_id, 'prev':prev, 'next':nxt, 'content':text, 'who':who, 'year':year} for u_id, prev, nxt, text, who, year in batch]
        cur.executemany('INSERT INTO utterance_fts (content) values (:content)', data)
        cur.executemany('INSERT INTO utterance (id, prev, next, who, year) values (:id, :prev, :next, :who, :year)', data)
    
        conn.commit()
            
    # TODO: Add speaker metadata
    

In [ ]:
from collections import Counter

term_counter = Counter()

with sqlite3.connect(tmp_db) as conn:
    cur = conn.cursor()

    
    print(cur.execute('select count(*) from utterance_fts where content match "kvinna AND kvinnor"').fetchall())
    print(cur.execute('select count(*) from utterance_fts where content match "kvinna"').fetchall())
    print(cur.execute('select count(*) from utterance_fts where content match "kvinnor"').fetchall())
    print(cur.execute('select count(*) from utterance_fts where content match "kvinna OR kvinnor"').fetchall())
    print(cur.execute('select count(*) from utterance_fts where content match "kvinn*"').fetchall())

    term_counter = Counter((token.lower() for content in cur.execute('select content from utterance_fts where content match "kvinn*"')
        for token in content[0].split() if token.lower().startswith('kvinn')))
            
   


In [ ]:
term_counter.most_common()

In [ ]:
def get_timeline(match_pattern: str) -> List[Tuple[int, int]]:
    with sqlite3.connect(tmp_db) as conn:
        cur = conn.cursor()
    
        return cur.execute(f'select year, count(*) from utterance as u join utterance_fts as uf on u.rowid == uf.rowid where content match "{match_pattern}" group by year').fetchall()


In [ ]:
def get_or_timeline(keywords : List[str]) -> List[Tuple[int, int]]:
    pattern = ' OR '.join(keywords)
    return get_timeline(pattern)

In [ ]:
kvinn = get_timeline('kvinn*')
kvinna = get_timeline('kvinna')
korv = get_timeline('korv')

In [ ]:
def make_line(timeline, name):
    x,y = zip(*timeline)
    return go.Line(x=x, y=y, name=name)
    

In [ ]:
"""
    The first element is the name of the line -- what shows up in the legend of the plot.
        "Name" : ['search', 'terms', 'in', 'any', 'order'],
        "Name" : ['also', 'called', 'keywords'],
    The second element is a `list` of searchterms.

    Note that the `name` and `keywords` / `search terms` are in quotes:
        '
        "
    You can use either, but they have to match:
       "This is a valid string"
       'This is another valid string'
       "but this will not work' 

    In the list (denoted by square brackets "[]") commas "," separate different items in the list.
        ['one', 'two' "three"] == ["one", 'twothree'] 

    The comma at the end of the lines " ], " are used to differentiate between the different lines.
    
"""
queries = {
    "kvinn*" : ['kvinn*'],
    "kvinna" : ['kvinna'],
    'The name that shows up in the plot' : ['term1', 'term_2', 'ansikte', 'fler', 'etc']
}
lines = [make_line(get_or_timeline(keywords), name) for name, keywords in queries.items()]
pyo.iplot(lines)